In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from sklearn.metrics.pairwise import cosine_similarity
from transformers import TFBertModel, BertTokenizer


## Langkah-langkah untuk Membangun Sistem Rekomendasi Resep dengan Jaringan Siamese
Notebook ini bertujuan untuk membuat sistem rekomendasi resep menggunakan pendekatan jaringan Siamese. Berikut adalah tahapan-tahapan yang akan kita lakukan dalam proses ini:

- Memuat dan Memproses Dataset: Menggabungkan kolom-kolom penting seperti kategori, judul, dan bahan untuk menjadi satu teks yang nantinya akan diproses oleh model BERT.

- Membuat Embedding BERT: Menggunakan model BERT yang sudah dilatih sebelumnya untuk menghasilkan embedding dari setiap resep, yang akan merepresentasikan setiap resep dalam bentuk vektor.

- Membuat Pasangan Positif dan Negatif: Berdasarkan kategori, kita membuat pasangan positif (kategori sama) dan negatif (kategori berbeda) sebagai data latih untuk model.

- Mendefinisikan dan Melatih Jaringan Siamese: Membangun dan melatih jaringan Siamese dengan menggunakan pasangan positif dan negatif. Model ini dilatih menggunakan contrastive loss untuk mengoptimalkan jarak antara resep yang mirip dan yang tidak mirip.

- Menyimpan Model dan Embedding: Setelah pelatihan selesai, kita simpan model dan embedding resep sehingga bisa digunakan lagi tanpa harus melatih model atau menghitung ulang embedding setiap kali.

Setelah proses selesai, sistem ini dapat memberikan rekomendasi resep yang mirip berdasarkan resep-resep yang sudah disimpan.

In [3]:
recipes_df = pd.read_csv('full_format_recipes.csv')

Pertama-tama, kita muat dataset dan menggabungkan beberapa kolom seperti categories, title, dan ingredients menjadi satu teks. Ini membantu kita membuat representasi teks yang lebih lengkap untuk setiap resep, sehingga BERT bisa menghasilkan embedding yang lebih baik.



In [ ]:
def preprocess_text(row):
    categories = ' '.join(eval(row['categories'])) if pd.notnull(row['categories']) else ''
    ingredients = ' '.join(eval(row['ingredients'])) if pd.notnull(row['ingredients']) else ''
    return f"{categories}. {ingredients}"

# Menghapus Fitur Data ['title'] untuk membuat model similiarity lebih spread out

recipes_df['bert_input_text'] = recipes_df.apply(preprocess_text, axis=1)
texts = recipes_df['bert_input_text'].tolist()

## Membuat Embedding BERT
Setelah teks untuk setiap resep siap, kita menggunakan model BERT untuk menghasilkan embedding. Embedding ini akan digunakan sebagai dasar untuk melatih jaringan siamese, yang akan belajar mengenali kemiripan antar resep.

In [8]:
bert_model = TFBertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [9]:
bert_embeddings = []
for text in texts:
    inputs = tokenizer(text, return_tensors="tf", padding=True, truncation=True)
    outputs = bert_model(inputs)
    cls_embedding = outputs.last_hidden_state[:, 0, :]  
    bert_embeddings.append(cls_embedding.numpy().flatten()) 

bert_embeddings = np.array(bert_embeddings)

In [10]:
recipes_df['embedding'] = list(bert_embeddings)
embeddings = recipes_df['embedding']
embeddings = np.array(list(recipes_df['embedding'])).reshape(-1, 768, 1)


## Membuat Pasangan Positif dan Negatif
Pada tahap ini, kita membuat pasangan positif dan negatif. Pasangan positif berisi resep-resep yang memiliki kategori serupa, sementara pasangan negatif memiliki kategori yang berbeda. Dengan pasangan-pasangan ini, model bisa belajar membedakan antara resep yang mirip dan tidak mirip.



In [11]:
def common_category(categories1, categories2):
    set1 = set(eval(categories1)) if isinstance(categories1, str) else set(categories1)
    set2 = set(eval(categories2)) if isinstance(categories2, str) else set(categories2)
    return len(set1.intersection(set2)) > 0

def generate_pairs(recipes_df, embeddings, num_pairs=1000):
    positive_pairs, negative_pairs = [], []
    
    for _ in range(num_pairs):
        idx1 = np.random.randint(len(recipes_df))
        
        positive_candidates = [idx for idx in recipes_df.index if idx != idx1 and common_category(
            recipes_df.iloc[idx1]['categories'], recipes_df.iloc[idx]['categories'])]
        
        if positive_candidates:
            idx2 = np.random.choice(positive_candidates)
            positive_pairs.append([embeddings[idx1], embeddings[idx2]])

        negative_candidates = [idx for idx in recipes_df.index if idx != idx1 and not common_category(
            recipes_df.iloc[idx1]['categories'], recipes_df.iloc[idx]['categories'])]
        
        if negative_candidates:
            idx3 = np.random.choice(negative_candidates)
            negative_pairs.append([embeddings[idx1], embeddings[idx3]])

    return np.array(positive_pairs), np.array(negative_pairs)

positive_pairs, negative_pairs = generate_pairs(recipes_df, embeddings)



## Mendefinisikan dan Melatih Model Jaringan Siamese
Di bagian ini, kita membangun jaringan Siamese. Jaringan ini memiliki arsitektur berbagi bobot (shared network), yang berarti kedua input resep (dalam pasangan positif atau negatif) akan diproses melalui jaringan yang sama. Model ini dilatih dengan menggunakan contrastive loss sehingga embedding dari resep yang mirip akan lebih dekat, sedangkan yang tidak mirip akan semakin berjauhan.

In [24]:

def contrastive_loss(y_true, y_pred, margin=1.0):
    positive_loss = y_true * tf.square(y_pred)
    negative_loss = (1 - y_true) * tf.square(tf.maximum(margin - y_pred, 0))
    return tf.reduce_mean(positive_loss + negative_loss)

input_shape = (768,)  
input_a = layers.Input(shape=input_shape)
input_b = layers.Input(shape=input_shape)

shared_network = tf.keras.Sequential([
    layers.InputLayer(shape=(768,)),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu')
])
processed_a = shared_network(input_a)
processed_b = shared_network(input_b)
distance = layers.Lambda(lambda embeddings: tf.norm(embeddings[0] - embeddings[1], axis=1, keepdims=True))([processed_a, processed_b])
siamese_model = Model(inputs=[input_a, input_b], outputs=distance)
siamese_model.compile(optimizer='adam', loss=contrastive_loss)


In [26]:
siamese_model.fit([positive_pairs[:, 0], positive_pairs[:, 1]], np.ones(len(positive_pairs)),
                  epochs=10, batch_size=32)

siamese_model.fit([negative_pairs[:, 0], negative_pairs[:, 1]], np.zeros(len(negative_pairs)),
                  epochs=10, batch_size=32)


Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0000e+00
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0000e+00
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0000e+00
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0000e+00
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0000e+00
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0000e+00
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0000e+00
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0000e+00
Epoch 9/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0000e+00
Epoch 10/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0000e+00
Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0000
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0000
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0000
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0000
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

## Menyimpan Model dan Embedding
Setelah selesai melatih model, kita simpan model yang sudah terlatih dan embedding yang telah dihasilkan. Dengan begitu, kita dapat menggunakan model ini untuk rekomendasi tanpa harus menghitung ulang embedding atau melatih ulang model setiap kali.

In [31]:
siamese_model.save("siamese_model_up.h5")

In [32]:
siamese_model.save("siamese_model_up.keras")

In [ ]:
recipes_df.to_csv("recipes_with_final_embeddings_up.csv", index=False)


In [43]:
np.savez("bert_embeddings_up.npz", bert_embeddings)

In [34]:
np.savez("pairs_data_up.npz", positive_pairs=positive_pairs, negative_pairs=negative_pairs)


In [35]:
loaded_data = np.load("pairs_data_up.npz")
positive_pairs = loaded_data['positive_pairs']
negative_pairs = loaded_data['negative_pairs']


In [36]:
print("Positive pairs sample:", positive_pairs[:5])
print("Negative pairs sample:", negative_pairs[:5])


Positive pairs sample: [[[[-0.76791894]
   [-0.2520941 ]
   [-0.13335492]
   ...
   [-0.88982743]
   [-0.00443599]
   [ 0.04973048]]

  [[-0.55208814]
   [ 0.05981527]
   [ 0.25554323]
   ...
   [-0.6429797 ]
   [-0.05616171]
   [ 0.17085686]]]


 [[[-0.66103214]
   [-0.30348742]
   [-0.3116385 ]
   ...
   [-0.19763482]
   [-0.01908185]
   [ 0.23871489]]

  [[-0.57389295]
   [-0.09762322]
   [ 0.02418999]
   ...
   [-0.40753457]
   [-0.32132417]
   [ 0.16687067]]]


 [[[-0.62901556]
   [-0.133837  ]
   [ 0.06293941]
   ...
   [-0.6795049 ]
   [ 0.00517223]
   [ 0.15844518]]

  [[-0.6297413 ]
   [-0.1843409 ]
   [-0.03472301]
   ...
   [-0.43930492]
   [-0.24457535]
   [ 0.06998876]]]


 [[[-0.66011465]
   [ 0.08648633]
   [ 0.18873326]
   ...
   [-0.46646905]
   [-0.19052383]
   [ 0.18789072]]

  [[-0.62304646]
   [-0.06366557]
   [ 0.23521498]
   ...
   [-0.6202389 ]
   [ 0.04591829]
   [ 0.11876895]]]


 [[[-0.7901331 ]
   [-0.19719501]
   [ 0.10059106]
   ...
   [-0.5001638 ]
   [-0

In [37]:
# Calculate distances between embeddings in positive pairs
pos_distances = [np.linalg.norm(pair[0] - pair[1]) for pair in positive_pairs]
neg_distances = [np.linalg.norm(pair[0] - pair[1]) for pair in negative_pairs]

print("Average distance for positive pairs:", np.mean(pos_distances))
print("Average distance for negative pairs:", np.mean(neg_distances))


Average distance for positive pairs: 6.665176
Average distance for negative pairs: 7.252311
